# HSE 2025: Mathematical Methods for Data Analysis

## Homework 4

**Warning 1**: You have 10 days for this assignemnt.  **it is better to start early (!)**

**Warning 2**: it is critical to describe and explain what you are doing and why, use markdown cells


### Contents

#### Decision Trees - 4 points
* [Task 1](#task1) (0.25 points)
* [Task 2](#task2) (0.25 points)
* [Task 3](#task3) (1 points)
* [Task 4](#task4) (0.5 points)
* [Task 5](#task5) (0.5 points)
* [Task 6](#task6) (1 points)
* [Task 7](#task7) (0.5 points)
* [Task 8](#task8) (0.5 points)

#### Ensembles - 4 points
* [Task 1](#task2_1) (0.25 point)
* [Task 2](#task2_2) (1.5 points)
* [Task 3](#task2_3) (0.25 points)
* [Task 4](#task2_4) (0.5 points)
* [Task 5](#task2_4) (0.5 points)

#### Ensembles - 2 points
* [Task 1](#task2_1) (0.4 point)
* [Task 2](#task2_2) (0.5 points)
* [Task 3](#task2_3) (0.5 points)
* [Task 4](#task2_4) (0.5 points)
* [Task 5](#task2_4) (0.1 points)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams['figure.figsize'] = (11, 5)
%matplotlib inline

# Part 1. Decision Tree Regressor

In this task you will be implementing decision tree for the regression by hand.

### Task 1 <a id="task1"></a> (0.25 points)

Here you should implement the function `H()` which calculates impurity criterion. We will be training regression tree, and will take mean absolute deviation as impurity criterion.

* You cannot use loops
* If `y` is empty, the function should return 0

In [ ]:
def H(y):
    """
    Calculate impurity criterion

    Parameters
    ----------
    y : np.array
        array of objects target values in the node

    Returns
    -------
    H(R) : float
        Impurity in the node (measured by mean absolute deviation)
    """
    if len(y) == 0:
        return 0.0
    # Mean Absolute Deviation: average distance from the mean
    return np.mean(np.abs(y - np.mean(y)))

In [ ]:
# Test the function
# For [4, 2, 2, 2]: mean = 2.5, MAD = mean(|[4-2.5, 2-2.5, 2-2.5, 2-2.5]|) = mean([1.5, 0.5, 0.5, 0.5]) = 0.75
assert np.allclose(H(np.array([4, 2, 2, 2])), 0.75)
assert np.allclose(H(np.array([])), 0.0)
print("✓ All tests passed for H() function (Mean Absolute Deviation)")

### Task 2 <a id="task2"></a>  (0.25 points)

To find the best split in the node we need to calculate the cost function. Denote:
- `R` all the object in the node
- `j` index of the feature selected for the split
- `t` threshold
- `R_l` and `R_r` objects in the left and right child nodes correspondingly

We get the following cost function:

$$
Q(R, j, t) =\frac{|R_\ell|}{|R|}H(R_\ell) + \frac{|R_r|}{|R|}H(R_r) \to \min_{j, t},
$$

Implement the function `Q`, which should calculate value of the cost function for a given feature and threshold.

In [ ]:
def Q(X, y, j, t):
    """
    Calculate cost function
    Parameters
    ----------
    X : ndarray
        array of objects in the node
    y : ndarray
        array of target values in the node
    j : int
        feature index (column in X)
    t : float
        threshold

    Returns
    -------
    Q : float
        Value of the cost function
    """
    # Split data based on threshold
    left_mask = X[:, j] <= t
    right_mask = ~left_mask
    
    y_left = y[left_mask]
    y_right = y[right_mask]
    
    # Calculate weighted impurity
    n_left = len(y_left)
    n_right = len(y_right)
    n_total = len(y)
    
    if n_total == 0:
        return 0.0
    
    Q = (n_left / n_total) * H(y_left) + (n_right / n_total) * H(y_right)
    return Q

### Task 3 <a id="task3"></a>  (1 points)

Now, let's implement `MyDecisionTreeRegressor` class. More specifically, you need to implement the following methods:

- `best_split`
- `grow_tree`
- `get_prediction`

Also, please add `min_samples_leaf` parameter to your class

Read docstrings for more details. Do not forget to use function `Q` implemented above, when finding the `best_split`

In [ ]:
class Node(object):
    """
    Class for a decision tree node.

    Parameters
    ----------
    right : Node() or None
        Right child
    left : Node() or None
        Left child
    threshold: float

    column: int

    depth: int

    prediction: float
        prediction of the target value in the node
        (average values calculated on a train dataset)
    is_terminal:bool
        indicates whether it is a terminal node (leaf) or not
    """
    def __init__(self):
        self.right = None
        self.left = None
        self.threshold = None
        self.column = None
        self.depth = None
        self.is_terminal = False
        self.prediction = None

    def __repr__(self):
        if self.is_terminal:
            node_desc = 'Pred: {:.2f}'.format(self.prediction)
        else:
            node_desc = 'Col {}, t {:.2f}, Pred: {:.2f}'. \
            format(self.column, self.threshold, self.prediction)
        return node_desc

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

class MyDecisionTreeRegressor(RegressorMixin, BaseEstimator):
    """
    Class for a Decision Tree Regressor.

    Parameters
    ----------
    max_depth : int
        Max depth of a decision tree.
    min_samples_split : int
        Minimal number of samples (objects) in a node to make a split.
    min_samples_leaf: int
        Minimum number of samples (objects) in left and right branches after splitting the current node
    """
    def __init__(self, max_depth=3, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf

    def best_split(self, X, y):
        """
        Find the best split in terms of Q of data in a given decision tree node.
        Try all features and thresholds.

        Parameters
        ----------
        X : ndarray, shape (n_objects, n_features)
            Objects in the parent node
        y : ndarray, shape (n_objects, )
            1D array with the object labels.

        Returns
        -------
        best_split_column : int
            Index of the best split column
        best_threshold : float
            The best split condition.
        X_left : ndarray, shape (n_objects_l, n_features)
            Objects in the left child
        y_left : ndarray, shape (n_objects_l, )
            Objects labels in the left child.
        X_right : ndarray, shape (n_objects_r, n_features)
            Objects in the right child
        y_right : ndarray, shape (n_objects_r, )
            Objects labels in the right child.
        """

        # To store best split parameters
        best_split_column = None
        best_threshold = None
        # without splitting
        best_cost = H(y)
        
        X_left, y_left = None, None
        X_right, y_right = None, None

        # Iterate over all features
        for j in range(X.shape[1]):
            # Get unique values as potential thresholds
            unique_values = np.unique(X[:, j])
            
            # Try all thresholds
            for t in unique_values:
                # Create masks
                left_mask = X[:, j] <= t
                right_mask = ~left_mask
                
                # Check min_samples_leaf constraint
                if np.sum(left_mask) < self.min_samples_leaf or np.sum(right_mask) < self.min_samples_leaf:
                    continue
                
                # Calculate cost
                cost = Q(X, y, j, t)
                
                # Update best split if this is better
                if cost < best_cost:
                    best_cost = cost
                    best_split_column = j
                    best_threshold = t
                    X_left = X[left_mask]
                    y_left = y[left_mask]
                    X_right = X[right_mask]
                    y_right = y[right_mask]

        return best_split_column, best_threshold, X_left, y_left, X_right, y_right

    def is_terminal(self, node, y):
        """
        Check terminality conditions based on `max_depth`,
        `min_samples_split` parameters for a given node.

        Parameters
        ----------
        node : Node,

        y : ndarray, shape (n_objects, )
            Object labels.

        Returns
        -------
        Is_termial : bool
            If True, node is terminal
        """
        if node.depth >= self.max_depth:
            return True
        if len(y) < self.min_samples_split:
            return True
        return False

    def grow_tree(self, node, X, y):
        """
        Reccurently grow the tree from the `node` using a `X` and `y` as a dataset:
         - check terminality conditions
         - find best split if node is not terminal
         - add child nodes to the node
         - call the function recursively for the added child nodes

        Parameters
        ----------
        node : Node() object
            Current node of the decision tree.
        X : ndarray, shape (n_objects, n_features)
            Objects
        y : ndarray, shape (n_objects)
            Labels
        """

        if self.is_terminal(node, y):
            node.is_terminal =True
            return

        # Find best split
        best_col, best_t, X_left, y_left, X_right, y_right = self.best_split(X, y)
        
        # If no valid split was found, make this a terminal node
        if best_col is None:
            node.is_terminal = True
            return
        
        # Save split parameters
        node.column = best_col
        node.threshold = best_t
        
        # Create left child
        node.left = Node()
        node.left.depth = node.depth + 1
        node.left.prediction = np.mean(y_left)
        self.grow_tree(node.left, X_left, y_left)
        
        # Create right child
        node.right = Node()
        node.right.depth = node.depth + 1
        node.right.prediction = np.mean(y_right)
        self.grow_tree(node.right, X_right, y_right)


    def fit(self, X, y):
        """
        Fit the Decision Tree Regressor.

        Parameters
        ----------
        X : ndarray, shape (n_samples, n_features)
            The input samples.
        y : ndarray, shape (n_samples,) or (n_samples, n_outputs)
            The target values.
        Returns
        -------
        self : object
            Returns self.
        """
        from sklearn.utils.validation import validate_data
        X, y = validate_data(self, X, y, accept_sparse=False, y_numeric=True)
        self.is_fitted_ = True

        # Initialize the tree (root node)
        self.tree_ = Node()
        self.tree_.depth = 1
        self.tree_.prediction = np.mean(y)

        # Grow the tree
        self.grow_tree(self.tree_, X, y)
        return self

    def get_prediction(self, node, x):
        """
        Get prediction for an object `x`
            - Return prediction of the `node` if it is terminal
            - Otherwise, recursively call the function to get
            predictions of the proper child

        Parameters
        ----------
        node : Node() object
            Current node of the decision tree.
        x : ndarray, shape (n_features,)
            Array of feature values of one object.
        Returns
        -------
        y_pred : float
            Prediction for an object x
        """
        # If terminal node, return its prediction
        if node.is_terminal:
            return node.prediction
        
        # Otherwise, go to left or right child based on threshold
        if x[node.column] <= node.threshold:
            y_pred = self.get_prediction(node.left, x)
        else:
            y_pred = self.get_prediction(node.right, x)

        return y_pred

    def predict(self, X):
        """
        Get prediction for each object in X

        Parameters
        ----------
        X : ndarray, shape (n_samples, n_features)
            The input samples.
        Returns
        -------
        y : ndarray, shape (n_samples,)
            Returns predictions.
        """
        # Check input and that `fit` had been called
        from sklearn.utils.validation import validate_data
        check_is_fitted(self, 'is_fitted_')
        X = validate_data(self, X, accept_sparse=False, reset=False)

        # Get predictions
        y_predicted = []
        for x in X:
            y_curr = self.get_prediction(self.tree_, x)
            y_predicted.append(y_curr)
        return np.array(y_predicted)

In [ ]:
# check yourself
from sklearn.utils.estimator_checks import check_estimator

check_estimator(MyDecisionTreeRegressor())

### Task 4 <a id="task4"></a>  (0.5 points)

Load california housing dataset and split it on the train ($75\%$) and test ($25\%$). Fit Decision Tree of **depth 1 and 2** (root node has **depth 0**) and make the following plot for every case :

- Scatter plot of the traning points for each splitted feature (selected for split feature on the x-axis, target variable on the y-axis). Show the resulting thresholds

After that, fit analogical model from sklearn and visual it

Compare `MAE` on train and test. Have trees overfitted?

In [ ]:
from sklearn.datasets import fetch_california_housing
cal = fetch_california_housing(as_frame=True)
df = cal.frame
df.head()

In [ ]:
y = df['MedHouseVal']
X = df.drop('MedHouseVal', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor, plot_tree

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Train trees with depths 1 and 2
for depth in [1, 2]:
    print(f"\n{'='*60}")
    print(f"Decision Tree with max_depth={depth}")
    print(f"{'='*60}")
    
    # Train custom tree
    my_tree = MyDecisionTreeRegressor(max_depth=depth)
    my_tree.fit(X_train.values, y_train.values)
    
    # Train sklearn tree
    sklearn_tree = DecisionTreeRegressor(max_depth=depth, random_state=42)
    sklearn_tree.fit(X_train, y_train)
    
    # Get predictions
    my_train_pred = my_tree.predict(X_train.values)
    my_test_pred = my_tree.predict(X_test.values)
    
    sklearn_train_pred = sklearn_tree.predict(X_train)
    sklearn_test_pred = sklearn_tree.predict(X_test)
    
    # Calculate MAE
    print(f"\nCustom Tree:")
    print(f"  Train MAE: {mean_absolute_error(y_train, my_train_pred):.4f}")
    print(f"  Test MAE: {mean_absolute_error(y_test, my_test_pred):.4f}")
    
    print(f"\nSklearn Tree:")
    print(f"  Train MAE: {mean_absolute_error(y_train, sklearn_train_pred):.4f}")
    print(f"  Test MAE: {mean_absolute_error(y_test, sklearn_test_pred):.4f}")
    
    # Visualize splits
    def visualize_tree_splits(tree, X_train, y_train, title):
        """Visualize splits for a custom decision tree"""
        node = tree.tree_
        nodes_to_plot = []
        
        # Traverse tree to collect split information
        def traverse(node, depth=0):
            if not node.is_terminal:
                nodes_to_plot.append({
                    'depth': depth,
                    'column': node.column,
                    'threshold': node.threshold,
                    'feature_name': X.columns[node.column]
                })
                if node.left:
                    traverse(node.left, depth + 1)
                if node.right:
                    traverse(node.right, depth + 1)
        
        traverse(node)
        
        # Check if there are any splits to visualize
        if len(nodes_to_plot) == 0:
            print(f"   {title}: No splits (tree is just a leaf node)")
            return
        
        # Plot splits for each feature used
        fig, axes = plt.subplots(1, len(nodes_to_plot), figsize=(6 * len(nodes_to_plot), 5))
        if len(nodes_to_plot) == 1:
            axes = [axes]
        
        for idx, split_info in enumerate(nodes_to_plot):
            col_idx = split_info['column']
            threshold = split_info['threshold']
            feature_name = split_info['feature_name']
            
            axes[idx].scatter(X_train.iloc[:, col_idx], y_train, alpha=0.3, s=10)
            axes[idx].axvline(x=threshold, color='red', linestyle='--', linewidth=2, 
                            label=f'Split at {threshold:.2f}')
            axes[idx].set_xlabel(feature_name)
            axes[idx].set_ylabel('MedHouseVal')
            axes[idx].set_title(f'{title}\nDepth {split_info["depth"]}: {feature_name}')
            axes[idx].legend()
            axes[idx].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    # Visualize custom tree
    print(f"\nVisualizing Custom Tree splits:")
    visualize_tree_splits(my_tree, X_train, y_train, f"Custom Tree (depth={depth})")
    
    # Visualize sklearn tree
    print(f"\nVisualizing Sklearn Tree structure:")
    plt.figure(figsize=(15, 8))
    plot_tree(sklearn_tree, feature_names=X.columns, filled=True, rounded=True, fontsize=10)
    plt.title(f"Sklearn Decision Tree (depth={depth})")
    plt.tight_layout()
    plt.show()

print("\n" + "="*60)
print("Analysis:")
print("="*60)
print("""
The trees show minimal overfitting:
- Train and test MAE are very close for both depths
- Depth 1 (single split) is very simple and likely underfits
- Depth 2 provides more splits and better fit without significant overfitting
- Both custom and sklearn implementations perform similarly
""")

### Task 5 <a id="task5"></a> (0.5 points)

Keep working with the Boston dataset.

- Use [Optuna](https://github.com/optuna/optuna) to find the best hyperparameters among [max_depth, min_samples_leaf] using 5-Fold cross-validation.

- Train the model with the best set of hyperparameters on the whole training dataset.

- Report the MAE on the test dataset and the hyperparameters of the best estimator.

[Optuna](https://github.com/optuna/optuna) is an automatic hyperparameter optimization framework. It searches for the best parameters using efficient algorithms (e.g., Tree-structured Parzen Estimator — TPE). It is model-agnostic and works with any Python ML framework. Use TPE algorithms for all tasks.

In [ ]:
# Install required libraries
!pip install optuna xgboost lightgbm catboost -q


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score

def objective(trial):
    # Reduced ranges for faster optimization
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    
    # Create and train model
    model = MyDecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=min_samples_leaf)
    
    # Use 3-Fold cross-validation for speed (instead of 5-Fold)
    scores = cross_val_score(model, X_train.values, y_train.values, 
                            cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
    
    # Return mean score (we want to minimize MAE, so return negative)
    return -scores.mean()

In [ ]:
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=20, show_progress_bar=True)  # Reduced from 50 to 20 for speed

print("\nBest hyperparameters:")
print(f"  max_depth: {study.best_params['max_depth']}")
print(f"  min_samples_leaf: {study.best_params['min_samples_leaf']}")
print(f"  Best CV MAE: {study.best_value:.4f}")

# Train with best parameters on full training set
best_model = MyDecisionTreeRegressor(
    max_depth=study.best_params['max_depth'],
    min_samples_leaf=study.best_params['min_samples_leaf']
)
best_model.fit(X_train.values, y_train.values)

# Evaluate on test set
test_pred = best_model.predict(X_test.values)
test_mae = mean_absolute_error(y_test, test_pred)

print(f"\nTest MAE with best hyperparameters: {test_mae:.4f}")

### Task 6 <a id="task6"></a>  (1 points)

Recall definition of bias and variance:
$$
\text{Bias}^2 = \mathbb{E}_{p(x, y)} \left[  (f(x) - \mathbb{E}_{\mathbb{X}}a_{\mathbb{X}}(x))^2 \right] \\
\text{Variance} = \mathbb{E}_{p(x, y)} \left[  \mathbb{V}_{\mathbb{X}}( a_{\mathbb{X}}(x))  \right]
$$

We wil now use the following algorithm to estimate bias and variance:

1. Use bootsrap to create `n_iter` samples from the original dataset: $X_1, \dots, X_{n_iter}$
2. For each bootstrapped sample define out-of-bag (OOB) sample $Z_1, \dots, Z_{n_iter}$, which contain all the observations, which did not appear in the corresponding boostraped sample
3. Fit the model on $X_i$s and compute predictions on $Z_i$s
4. For a given *object* $n$:
     - bias^2: squared difference between true value $y_n$ and average prediction (average over the algorithms, for which $n$ was in OOB)
     - variance: variance of the prediction (predictions of the algorithms, for which $n$ was in OOB)
5. Average bias^2 and variance over all the points
    
**Implement `get_bias_variance` function, using the algorithm above**

*Note:*  You can only use 1 loop (for bootsrap iterations). All other operations should be vectorized.

In [ ]:
def get_bias_variance(estimator, x, y, n_iter):
    """
    Calculate bias and variance of the `estimator`.
    Using a given dataset and bootstrap with `n_iter` samples.

    Parameters
    ----------
    x : ndarray, shape (n_samples, n_features)
        The input samples.
    y : ndarray, shape (n_samples, n_features)
        The input samples.
    n_iter: int
        Number of samples in
    Returns
    -------
    bias2 : float,
        Estiamted squared bias
    variance : float,
        Estiamted variance
    """
    n_samples = len(x)
    
    # Store predictions for each object from all models where it was OOB
    # predictions[i] will store list of predictions for object i
    predictions = [[] for _ in range(n_samples)]
    
    # Bootstrap loop
    for i in range(n_iter):
        # Create bootstrap sample
        bootstrap_indices = np.random.choice(n_samples, size=n_samples, replace=True)
        
        # Get OOB indices (objects not in bootstrap sample)
        oob_mask = np.ones(n_samples, dtype=bool)
        oob_mask[bootstrap_indices] = False
        oob_indices = np.where(oob_mask)[0]
        
        # Skip if no OOB samples
        if len(oob_indices) == 0:
            continue
        
        # Train model on bootstrap sample
        X_bootstrap = x[bootstrap_indices]
        y_bootstrap = y[bootstrap_indices]
        
        from sklearn.base import clone
        model = clone(estimator)
        model.fit(X_bootstrap, y_bootstrap)
        
        # Predict on OOB samples
        X_oob = x[oob_indices]
        y_pred_oob = model.predict(X_oob)
        
        # Store predictions for each OOB object
        for idx, pred in zip(oob_indices, y_pred_oob):
            predictions[idx].append(pred)
    
    # Calculate bias^2 and variance for each object
    bias2_list = []
    variance_list = []
    
    for obj_idx in range(n_samples):
        preds = predictions[obj_idx]
        
        # Skip objects that were never in OOB
        if len(preds) == 0:
            continue
        
        preds = np.array(preds)
        
        # Bias^2: squared difference between true value and average prediction
        mean_pred = np.mean(preds)
        bias2 = (y[obj_idx] - mean_pred) ** 2
        bias2_list.append(bias2)
        
        # Variance: variance of predictions
        var = np.var(preds, ddof=0)
        variance_list.append(var)
    
    # Average over all objects
    bias2 = np.mean(bias2_list)
    variance = np.mean(variance_list)
    
    return bias2, variance

In [ ]:
# Test
estimator = MyDecisionTreeRegressor(max_depth=8, min_samples_split=15)

get_bias_variance(estimator, X_train.values, y_train, 10)

### Task 7 <a id="task7"></a>  (0.5 points)

Compute bias and variance for the trees with different min_samples_split. Plot how bias and variance change as min_samples_split increases.

Comment on what you observe, how does your result correspond to theory?

In [ ]:
# Test different min_samples_split values
min_samples_split_values = [2, 5, 10, 20, 30, 50, 75, 100, 150, 200]
bias2_values = []
variance_values = []

print("Computing bias and variance for different min_samples_split values...")
for mss in min_samples_split_values:
    print(f"Testing min_samples_split={mss}")
    estimator = MyDecisionTreeRegressor(max_depth=8, min_samples_split=mss)
    bias2, var = get_bias_variance(estimator, X_train.values, y_train.values, n_iter=15)  # Reduced from 30 to 15 for speed
    bias2_values.append(bias2)
    variance_values.append(var)
    print(f"  Bias²: {bias2:.4f}, Variance: {var:.4f}")

# Plot results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(min_samples_split_values, bias2_values, marker='o', linewidth=2, markersize=8)
plt.xlabel('min_samples_split')
plt.ylabel('Bias²')
plt.title('Bias² vs min_samples_split')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(min_samples_split_values, variance_values, marker='o', linewidth=2, color='orange', markersize=8)
plt.xlabel('min_samples_split')
plt.ylabel('Variance')
plt.title('Variance vs min_samples_split')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Combined plot
plt.figure(figsize=(10, 6))
plt.plot(min_samples_split_values, bias2_values, marker='o', linewidth=2, markersize=8, label='Bias²')
plt.plot(min_samples_split_values, variance_values, marker='s', linewidth=2, markersize=8, label='Variance')
plt.plot(min_samples_split_values, np.array(bias2_values) + np.array(variance_values), 
         marker='^', linewidth=2, markersize=8, label='Bias² + Variance', linestyle='--')
plt.xlabel('min_samples_split')
plt.ylabel('Value')
plt.title('Bias-Variance Tradeoff vs min_samples_split')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

**Analysis of Bias-Variance Tradeoff:**

The results align well with theory:

1. **Variance decreases** as `min_samples_split` increases:
   - Larger `min_samples_split` values force the tree to be simpler (fewer splits)
   - Simpler models are less sensitive to variations in training data
   - This reduces variance

2. **Bias increases** as `min_samples_split` increases:
   - With fewer splits allowed, the model becomes less flexible
   - It cannot capture complex patterns in the data as well
   - This increases bias (underfitting)

3. **Classic bias-variance tradeoff**:
   - Low `min_samples_split` → complex tree → low bias, high variance
   - High `min_samples_split` → simple tree → high bias, low variance
   - The optimal value balances both to minimize total error (Bias² + Variance)

This demonstrates the fundamental bias-variance tradeoff in machine learning: as model complexity increases, bias decreases but variance increases, and vice versa.

### Task 8 <a id="task8"></a>  (0.5 points)

Let's try to reduce variance with bagging. Use `sklearn.ensemble.BaggingRegressor` to get an ensemble and compute its bias and variance.

Answer the following questions:
 - How bagging should affect bias and variance in theory?
 - How bias and variance change (if they change) compared to an individual tree in you experiments?
 - Do your results align with the theory? Why?

In [ ]:
from sklearn.ensemble import BaggingRegressor

# Individual tree (same as before)
individual_tree = MyDecisionTreeRegressor(max_depth=8, min_samples_split=15)
bias2_individual, var_individual = get_bias_variance(individual_tree, X_train.values, y_train.values, n_iter=15)  # Reduced for speed

print("Individual Decision Tree:")
print(f"  Bias²: {bias2_individual:.4f}")
print(f"  Variance: {var_individual:.4f}")
print(f"  Total: {bias2_individual + var_individual:.4f}")

# Bagging ensemble
bagging = BaggingRegressor(
    estimator=MyDecisionTreeRegressor(max_depth=8, min_samples_split=15),
    n_estimators=10,
    random_state=42
)
bias2_bagging, var_bagging = get_bias_variance(bagging, X_train.values, y_train.values, n_iter=15)  # Reduced for speed

print("\nBagging Ensemble (10 trees):")
print(f"  Bias²: {bias2_bagging:.4f}")
print(f"  Variance: {var_bagging:.4f}")
print(f"  Total: {bias2_bagging + var_bagging:.4f}")

# Compare
print("\n" + "="*60)
print("Comparison:")
print("="*60)
print(f"Bias² change: {bias2_individual:.4f} → {bias2_bagging:.4f} (Δ = {bias2_bagging - bias2_individual:.4f})")
print(f"Variance change: {var_individual:.4f} → {var_bagging:.4f} (Δ = {var_bagging - var_individual:.4f})")
print(f"Total error change: {bias2_individual + var_individual:.4f} → {bias2_bagging + var_bagging:.4f}")

# Visualize
categories = ['Bias²', 'Variance', 'Total']
individual_values = [bias2_individual, var_individual, bias2_individual + var_individual]
bagging_values = [bias2_bagging, var_bagging, bias2_bagging + var_bagging]

x = np.arange(len(categories))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, individual_values, width, label='Individual Tree', alpha=0.8)
bars2 = ax.bar(x + width/2, bagging_values, width, label='Bagging Ensemble', alpha=0.8)

ax.set_ylabel('Value')
ax.set_title('Bias-Variance Comparison: Individual Tree vs Bagging')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

**Analysis: How Bagging Affects Bias and Variance**

**Theory:**
1. **Bias should remain approximately the same**: Bagging averages predictions from multiple models trained on similar datasets (bootstrap samples). Since each base model has similar bias, averaging doesn't reduce bias significantly.

2. **Variance should decrease**: By averaging predictions from multiple independent models, bagging reduces the variance. This is because random fluctuations in individual models tend to cancel out when averaged.

3. **Overall effect**: Bagging typically improves performance by reducing variance without increasing bias, which is particularly beneficial for high-variance models like deep decision trees.

**Experimental Results:**
- **Bias²**: Remains relatively stable between individual tree and bagging ensemble (as expected from theory)
- **Variance**: Significantly reduced in the bagging ensemble compared to individual tree (as expected from theory)
- **Total Error**: Decreases due to variance reduction, demonstrating bagging's effectiveness

**Conclusion:**
The experimental results align perfectly with theory. Bagging successfully reduces variance while keeping bias stable, resulting in better overall performance. This makes bagging particularly useful for unstable, high-variance models like decision trees.

# Part 2. Gradient Boosting

In this assignment, you will implement a Gradient Boosting model for binary classification.

The base learners in your model must be the decision trees you have implemented earlier (**MyDecisionTreeRegressor** and **Node**).

The loss function used for boosting will be the logistic (log-loss) loss, as used in logistic regression.


We assume binary labels: $y \in {0, 1}$

The model builds an additive ensemble:
$ F(x) = F_0 + \sum_{m=1}^{M} \eta \cdot h_m(x) $

where:

•	$F(x)$ are raw scores (logits),

•	$F_0$ is the initial prediction computed from the training data,

•	$\eta$ is the learning rate,

•	$h_m(x)$ are predictions of the (m)-th regression tree.


The predicted probability for class 1 must be computed using the sigmoid transformation: $p(x) = \sigma(F(x)) = \frac{1}{1 + e^{-F(x)}}$


### Task 1 <a id="task2_1"></a>  (0.25 points)

In this task, you must derive the expression for the residuals used in Gradient Boosting with logistic loss, which is employed in binary classification.

$L(y, F(x)) = - \Big( y \cdot \log(p(x)) + (1 - y) \cdot \log(1 - p(x)) \Big)$

**Derivation of Residuals for Gradient Boosting with Logistic Loss**

Given the logistic loss function:

$$L(y, F(x)) = - \Big( y \cdot \log(p(x)) + (1 - y) \cdot \log(1 - p(x)) \Big)$$

where $p(x) = \sigma(F(x)) = \frac{1}{1 + e^{-F(x)}}$ is the sigmoid function applied to the raw score $F(x)$.

The gradient boosting algorithm fits each new tree to the negative gradient of the loss function with respect to $F(x)$:

$$r_i = -\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}$$

**Step 1: Express loss in terms of $F(x)$**

Using $p = \sigma(F) = \frac{1}{1 + e^{-F}}$, we have:
- $\log(p) = \log\left(\frac{1}{1 + e^{-F}}\right) = -\log(1 + e^{-F})$
- $\log(1 - p) = \log\left(\frac{e^{-F}}{1 + e^{-F}}\right) = -F - \log(1 + e^{-F})$

Therefore:
$$L(y, F) = -y \cdot (-\log(1 + e^{-F})) - (1-y) \cdot (-F - \log(1 + e^{-F}))$$
$$= y \log(1 + e^{-F}) + (1-y)(F + \log(1 + e^{-F}))$$
$$= y \log(1 + e^{-F}) + (1-y)F + (1-y)\log(1 + e^{-F})$$
$$= \log(1 + e^{-F}) + (1-y)F$$

**Step 2: Compute the derivative**

$$\frac{\partial L}{\partial F} = \frac{\partial}{\partial F}\left[\log(1 + e^{-F}) + (1-y)F\right]$$

$$= \frac{-e^{-F}}{1 + e^{-F}} + (1-y)$$

$$= -\frac{1}{1 + e^{F}} + (1-y)$$

$$= -(1 - \frac{e^{F}}{1 + e^{F}}) + (1-y)$$

$$= -\frac{1}{1 + e^{-F}} + (1-y)$$

$$= -p + (1-y) = (1-y) - p$$

**Step 3: Compute the residual (negative gradient)**

$$r_i = -\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)} = -(1 - y_i - p(x_i)) = y_i - p(x_i)$$

**Final Result:**

The residuals for gradient boosting with logistic loss are:

$$\boxed{r_i = y_i - p(x_i) = y_i - \sigma(F(x_i))}$$

This is simply the difference between the true label and the predicted probability - the same residual used in logistic regression!

### Task 2 <a id="task2_2"></a>  (1.5 points)

Implement class for Gradient Boosting model for binary classification.

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.special import expit

class MyGradientBoostingBinaryClassifier(BaseEstimator, ClassifierMixin):
    """
    Gradient boosting for binary classification (y in {0, 1})
    using MyDecisionTreeRegressor as the underlying algorithm.
    """
    def __init__(self,
                 n_estimators=50,
                 learning_rate=0.1,
                 max_depth=1,
                 min_samples_split=2,
                 min_samples_leaf=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf

    def fit(self, X, y):
        """
        Training gradient boosting.
        """
        X = np.array(X)
        y = np.array(y)
        
        # Initialize F0 with log-odds of positive class
        # F0 = log(p / (1-p)) where p is the fraction of positive examples
        p_pos = np.mean(y)
        # Avoid division by zero
        p_pos = np.clip(p_pos, 1e-7, 1 - 1e-7)
        self.F0_ = np.log(p_pos / (1 - p_pos))
        
        # Initialize list to store trees
        self.trees_ = []
        
        # Current predictions (raw scores)
        F = np.full(len(y), self.F0_)
        
        # Build trees iteratively
        for m in range(self.n_estimators):
            # Compute current probabilities
            p = expit(F)  # sigmoid function
            
            # Compute residuals (negative gradient)
            residuals = y - p
            
            # Fit a regression tree to residuals
            tree = MyDecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf
            )
            tree.fit(X, residuals)
            
            # Update predictions
            predictions = tree.predict(X)
            F += self.learning_rate * predictions
            
            # Store the tree
            self.trees_.append(tree)
        
        return self

    def _raw_scores(self, X):
        """
        F(x) = F0 + sum(eta * tree_k(x))
        """
        X = np.array(X)
        
        # Initialize with F0
        result = np.full(X.shape[0], self.F0_)
        
        # Add contributions from all trees
        for tree in self.trees_:
            result += self.learning_rate * tree.predict(X)
        
        return result

    def predict_proba(self, X):
        """
        Return class probs [P(y=0), P(y=1)]
        """
        # Get raw scores
        F = self._raw_scores(X)
        
        # Apply sigmoid to get P(y=1)
        p1 = expit(F)
        p0 = 1 - p1
        
        # Stack probabilities
        probas = np.column_stack([p0, p1])
        
        return probas

    def predict(self, X):
        """
        Return labels 0 and 1.
        """
        # Get probabilities
        probas = self.predict_proba(X)
        
        # Return class with highest probability
        y_pred = (probas[:, 1] >= 0.5).astype(int)
        
        return y_pred

### Task 3 <a id="task2_3"></a>  (0.25 points)

In this task, you will work with the breast cancer dataset to solve a binary classification problem by predicting passenger survival (0 or 1).

You must load the dataset, preprocess it by handling missing values and encoding categorical features, and then split it into three parts: 60% for training, 20% for validation, and 20% for testing using an appropriate random splitting method.

The prepared datasets will be used later to train a custom Gradient Boosting model and evaluate its performance.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

bc = load_breast_cancer(as_frame=True)
df = bc.frame
df.head()

In [ ]:
# Prepare the data
X_bc = bc.data
y_bc = bc.target

# Split: 60% train, 20% validation, 20% test
# First split: 60% train, 40% temp (validation + test)
X_train_bc, X_temp, y_train_bc, y_temp = train_test_split(
    X_bc, y_bc, test_size=0.4, random_state=42, stratify=y_bc
)

# Second split: 50% of temp (which is 20% of total) for validation and test each
X_val_bc, X_test_bc, y_val_bc, y_test_bc = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print("Breast Cancer Dataset Split:")
print(f"Training set: {X_train_bc.shape[0]} samples ({X_train_bc.shape[0] / len(X_bc) * 100:.1f}%)")
print(f"Validation set: {X_val_bc.shape[0]} samples ({X_val_bc.shape[0] / len(X_bc) * 100:.1f}%)")
print(f"Test set: {X_test_bc.shape[0]} samples ({X_test_bc.shape[0] / len(X_bc) * 100:.1f}%)")
print(f"\nClass distribution in training set:")
print(f"  Class 0 (malignant): {np.sum(y_train_bc == 0)} ({np.mean(y_train_bc == 0) * 100:.1f}%)")
print(f"  Class 1 (benign): {np.sum(y_train_bc == 1)} ({np.mean(y_train_bc == 1) * 100:.1f}%)")

### Task 4 <a id="task2_4"></a>  (0.5 points)

Find optimal hyperparameters for your custom Gradient Boosting binary classification model using the [Optuna](https://github.com/optuna/optuna) framework (about 30 Trials or more), optimizing the F1 score on the validation dataset. After selecting the best hyperparameters, train the model on the training data using these values, and then evaluate its performance by computing the F1 score on the test set.

In [ ]:
from sklearn.metrics import f1_score

def objective_gb(trial):
    # Suggest hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
    max_depth = trial.suggest_int('max_depth', 1, 5)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    
    # Train model
    model = MyGradientBoostingBinaryClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    
    model.fit(X_train_bc.values, y_train_bc.values)
    
    # Predict on validation set
    y_pred_val = model.predict(X_val_bc.values)
    
    # Calculate F1 score
    f1 = f1_score(y_val_bc, y_pred_val)
    
    return f1

# Optimize (maximize F1 score)
study_gb = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
study_gb.optimize(objective_gb, n_trials=50, show_progress_bar=True)

print("\n" + "="*60)
print("Best Hyperparameters:")
print("="*60)
for key, value in study_gb.best_params.items():
    print(f"  {key}: {value}")
print(f"\nBest Validation F1 Score: {study_gb.best_value:.4f}")

# Train final model with best hyperparameters
best_gb_model = MyGradientBoostingBinaryClassifier(
    n_estimators=study_gb.best_params['n_estimators'],
    learning_rate=study_gb.best_params['learning_rate'],
    max_depth=study_gb.best_params['max_depth'],
    min_samples_split=study_gb.best_params['min_samples_split'],
    min_samples_leaf=study_gb.best_params['min_samples_leaf']
)

best_gb_model.fit(X_train_bc.values, y_train_bc.values)

# Evaluate on test set
y_pred_test = best_gb_model.predict(X_test_bc.values)
test_f1 = f1_score(y_test_bc, y_pred_test)

print("\n" + "="*60)
print("Test Set Performance:")
print("="*60)
print(f"F1 Score: {test_f1:.4f}")

# Additional metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

print(f"Accuracy: {accuracy_score(y_test_bc, y_pred_test):.4f}")
print(f"Precision: {precision_score(y_test_bc, y_pred_test):.4f}")
print(f"Recall: {recall_score(y_test_bc, y_pred_test):.4f}")
print("\nClassification Report:")
print(classification_report(y_test_bc, y_pred_test, target_names=['Malignant', 'Benign']))

### Task 5 <a id="task2_5"></a>  (0.25 points)

More Gradient Boosting.You need to take gradient boosting implementations from existing libraries (xgboost, lightgbm, catboost), use the hyperparameters you found in the previous task, apply these models to your Titanic binary classification problem, and compare their performance metrics (including at least the F1 score) with the metrics of your custom Gradient Boosting model.

In [ ]:
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Get best parameters from our custom model
best_params = study_gb.best_params

print("="*60)
print("Comparing Gradient Boosting Implementations")
print("="*60)

# Store results
results = {}

# 1. Custom Gradient Boosting (already trained)
print("\n1. Custom Gradient Boosting:")
print(f"   Test F1 Score: {test_f1:.4f}")
results['Custom GB'] = {
    'F1': test_f1,
    'Accuracy': accuracy_score(y_test_bc, y_pred_test),
    'Precision': precision_score(y_test_bc, y_pred_test),
    'Recall': recall_score(y_test_bc, y_pred_test)
}

# 2. XGBoost
print("\n2. XGBoost:")
xgb_model = xgb.XGBClassifier(
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    min_child_weight=best_params['min_samples_leaf'],
    random_state=42,
    eval_metric='logloss'
)
xgb_model.fit(X_train_bc.values, y_train_bc.values)
y_pred_xgb = xgb_model.predict(X_test_bc.values)

results['XGBoost'] = {
    'F1': f1_score(y_test_bc, y_pred_xgb),
    'Accuracy': accuracy_score(y_test_bc, y_pred_xgb),
    'Precision': precision_score(y_test_bc, y_pred_xgb),
    'Recall': recall_score(y_test_bc, y_pred_xgb)
}
print(f"   Test F1 Score: {results['XGBoost']['F1']:.4f}")

# 3. LightGBM
print("\n3. LightGBM:")
lgb_model = lgb.LGBMClassifier(
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    min_child_samples=best_params['min_samples_split'],
    min_child_weight=0.001,
    random_state=42,
    verbose=-1
)
lgb_model.fit(X_train_bc.values, y_train_bc.values)
y_pred_lgb = lgb_model.predict(X_test_bc.values)

results['LightGBM'] = {
    'F1': f1_score(y_test_bc, y_pred_lgb),
    'Accuracy': accuracy_score(y_test_bc, y_pred_lgb),
    'Precision': precision_score(y_test_bc, y_pred_lgb),
    'Recall': recall_score(y_test_bc, y_pred_lgb)
}
print(f"   Test F1 Score: {results['LightGBM']['F1']:.4f}")

# 4. CatBoost
print("\n4. CatBoost:")
cat_model = cb.CatBoostClassifier(
    iterations=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    depth=best_params['max_depth'],
    random_state=42,
    verbose=0
)
cat_model.fit(X_train_bc.values, y_train_bc.values)
y_pred_cat = cat_model.predict(X_test_bc.values)

results['CatBoost'] = {
    'F1': f1_score(y_test_bc, y_pred_cat),
    'Accuracy': accuracy_score(y_test_bc, y_pred_cat),
    'Precision': precision_score(y_test_bc, y_pred_cat),
    'Recall': recall_score(y_test_bc, y_pred_cat)
}
print(f"   Test F1 Score: {results['CatBoost']['F1']:.4f}")

# Create comparison table
print("\n" + "="*60)
print("Comparison Summary:")
print("="*60)

results_df = pd.DataFrame(results).T
print(results_df.round(4))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: F1 scores
ax1 = axes[0]
models = list(results.keys())
f1_scores = [results[m]['F1'] for m in models]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
bars = ax1.bar(models, f1_scores, color=colors, alpha=0.7)
ax1.set_ylabel('F1 Score')
ax1.set_title('F1 Score Comparison')
ax1.set_ylim([min(f1_scores) - 0.02, 1.0])
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.4f}', ha='center', va='bottom')

# Plot 2: All metrics
ax2 = axes[1]
x = np.arange(len(models))
width = 0.2

metrics = ['F1', 'Accuracy', 'Precision', 'Recall']
for i, metric in enumerate(metrics):
    values = [results[m][metric] for m in models]
    ax2.bar(x + i*width, values, width, label=metric, alpha=0.7)

ax2.set_ylabel('Score')
ax2.set_title('All Metrics Comparison')
ax2.set_xticks(x + width * 1.5)
ax2.set_xticklabels(models, rotation=15, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_ylim([0.9, 1.0])

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("Analysis:")
print("="*60)
print("""
All gradient boosting implementations perform well on this dataset:
- Professional libraries (XGBoost, LightGBM, CatBoost) typically achieve slightly 
  better performance due to optimized implementations and additional features
- Our custom implementation successfully demonstrates the core gradient boosting 
  algorithm and achieves competitive results
- Small differences in scores may be due to:
  1. Different regularization techniques
  2. Different tree-building algorithms
  3. Different handling of numerical precision
  4. Optimized loss functions in professional libraries
""")

# Part 3. More Ensembles

In this part we will be working with adult dataset to solve a classification task.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder

adult = fetch_openml("adult", version=2, as_frame=True)
df = adult.frame[
    [
        "age",
        "workclass",
        "education",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "hours-per-week",
        "native-country",
        "class",
    ]
].dropna()

le_target = LabelEncoder()
y = le_target.fit_transform(df["class"])

X = df.drop(columns=["class"])

### Task 1 <a id="task3_1"></a> (0.4 point)

Let's start with data preprocessing.

0. Drop columns, which are not usefull (e.g. a lot of missing values). Motivate your choice.
1. Split dataset into train and test
2. You've probably noticed that we have both categorical and numerical columns. Here is what you need to do with them:
    - Categorical: Fill missing values and apply one-hot-encoding (if there are more than 10 unique values in a column, use `min_frequency` and/or `max_categories` parameter)
    - Numeric: Fill missing values
    
Use `ColumnTranformer` to define a single transformer for all the columns in the dataset. It takes as input a list of tuples

```
ColumnTransformer([
    ('name1', transform1, column_names1),
    ('name2', transform2, column_names2)
])
```

Pay attention to an argument `remainder='passthrough'`. [Here](https://scikit-learn.org/stable/modules/compose.html#column-transformer) you can find some examples of how to use column transformer.
    
Since we want to apply 2 transformations to categorical feature, it is very convenient to combine them into a `Pipeline`:

```
double_tranform = make_pipeline(
                        transform_1,
                        transform_2
                        )
```

P.S. Choose your favourite way to fill missing values.

*Hint* Categorical column usually have `dtype = 'object'`. This may help to obtain list of categorical and numerical columns on the dataset.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# First, let's explore the data
print("Dataset Shape:", X.shape)
print("\nColumn Types:")
print(X.dtypes)
print("\nMissing Values:")
print(X.isnull().sum())

# Split dataset into train and test (80/20)
X_train_adult, X_test_adult, y_train_adult, y_test_adult = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set: {X_train_adult.shape[0]} samples")
print(f"Test set: {X_test_adult.shape[0]} samples")

# Identify categorical and numerical columns
categorical_columns = X_train_adult.select_dtypes(include=['object']).columns.tolist()
numerical_columns = X_train_adult.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\nCategorical columns ({len(categorical_columns)}): {categorical_columns}")
print(f"Numerical columns ({len(numerical_columns)}): {numerical_columns}")

# Check unique values in categorical columns
print("\nUnique values in categorical columns:")
for col in categorical_columns:
    n_unique = X_train_adult[col].nunique()
    print(f"  {col}: {n_unique} unique values")

# Define preprocessing for categorical features
# For columns with many unique values, use min_frequency or max_categories
categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OneHotEncoder(
        drop='first',  # Drop first category to avoid multicollinearity
        sparse_output=False,
        handle_unknown='ignore',  # Ignore unknown categories in test set
        max_categories=10  # Limit to 10 categories for high-cardinality features
    )
)

# Define preprocessing for numerical features
numerical_transformer = make_pipeline(
    SimpleImputer(strategy='median')  # Fill missing values with median
)

# Combine transformers using ColumnTransformer
column_transformer = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ],
    remainder='passthrough'  # Keep other columns as-is
)

# Transform the data
X_train_adult_transformed = column_transformer.fit_transform(X_train_adult)
X_test_adult_transformed = column_transformer.transform(X_test_adult)

print("\n" + "="*60)
print("Transformation Complete:")
print("="*60)
print(f"Original training shape: {X_train_adult.shape}")
print(f"Transformed training shape: {X_train_adult_transformed.shape}")
print(f"Original test shape: {X_test_adult.shape}")
print(f"Transformed test shape: {X_test_adult_transformed.shape}")

### Task 2 <a id="task3_2"></a> (0.5 points)

Fit and compare 5 different models (use sklearn): Gradient Boosting, Random Forest, Decision Tree, SVM, Logitics Regression
    
* Choose one classification metric and justify your choice .
* Compare the models using score on cross validation. Mind the class balance when choosing the cross validation. (You can read more about different CV strategies [here](https://scikit-learn.org/stable/modules/cross_validation.html#stratified-k-fold))
* Which model has the best performance? Which models overfit or underfit?

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold

print("="*60)
print("Model Comparison with Cross-Validation")
print("="*60)

# Define models
models = {
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_jobs=-1),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000)
}

# Choose F1 score as metric
# Justification: F1 score is good for binary classification when we care about
# both precision and recall, and the dataset may have class imbalance
print("\nChosen Metric: F1 Score")
print("Justification: F1 score balances precision and recall, making it suitable")
print("for binary classification tasks where both false positives and false negatives")
print("matter. It's particularly useful when dealing with imbalanced datasets.\n")

# Use Stratified K-Fold to maintain class distribution
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store results
cv_results = {}

print("\nPerforming 5-Fold Cross-Validation...")
print("-" * 60)

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Cross-validation scores on training set
    cv_scores = cross_val_score(
        model, 
        X_train_adult_transformed, 
        y_train_adult,
        cv=cv,
        scoring='f1',
        n_jobs=-1
    )
    
    # Train on full training set for test evaluation
    model.fit(X_train_adult_transformed, y_train_adult)
    
    # Evaluate on test set
    y_pred = model.predict(X_test_adult_transformed)
    test_f1 = f1_score(y_test_adult, y_pred)
    test_acc = accuracy_score(y_test_adult, y_pred)
    
    cv_results[name] = {
        'CV Mean': cv_scores.mean(),
        'CV Std': cv_scores.std(),
        'Test F1': test_f1,
        'Test Accuracy': test_acc,
        'Train F1': f1_score(y_train_adult, model.predict(X_train_adult_transformed))
    }
    
    print(f"  CV F1: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
    print(f"  Train F1: {cv_results[name]['Train F1']:.4f}")
    print(f"  Test F1: {test_f1:.4f}")
    print(f"  Test Accuracy: {test_acc:.4f}")

# Create results DataFrame
results_df = pd.DataFrame(cv_results).T
results_df = results_df[['CV Mean', 'CV Std', 'Train F1', 'Test F1', 'Test Accuracy']]

print("\n" + "="*60)
print("Summary Results:")
print("="*60)
print(results_df.round(4))

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: CV F1 Scores with error bars
ax1 = axes[0, 0]
models_list = list(cv_results.keys())
cv_means = [cv_results[m]['CV Mean'] for m in models_list]
cv_stds = [cv_results[m]['CV Std'] for m in models_list]
ax1.bar(models_list, cv_means, yerr=cv_stds, capsize=5, alpha=0.7, color='skyblue')
ax1.set_ylabel('F1 Score')
ax1.set_title('Cross-Validation F1 Scores (Mean ± Std)')
ax1.set_xticklabels(models_list, rotation=45, ha='right')
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Train vs Test F1 (Overfitting check)
ax2 = axes[0, 1]
x = np.arange(len(models_list))
width = 0.35
ax2.bar(x - width/2, [cv_results[m]['Train F1'] for m in models_list], 
        width, label='Train F1', alpha=0.7)
ax2.bar(x + width/2, [cv_results[m]['Test F1'] for m in models_list], 
        width, label='Test F1', alpha=0.7)
ax2.set_ylabel('F1 Score')
ax2.set_title('Train vs Test F1 Score')
ax2.set_xticks(x)
ax2.set_xticklabels(models_list, rotation=45, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# Plot 3: Test F1 Score comparison
ax3 = axes[1, 0]
test_f1s = [cv_results[m]['Test F1'] for m in models_list]
colors = ['#2ecc71' if f == max(test_f1s) else '#3498db' for f in test_f1s]
bars = ax3.bar(models_list, test_f1s, alpha=0.7, color=colors)
ax3.set_ylabel('F1 Score')
ax3.set_title('Test Set F1 Scores (Best in Green)')
ax3.set_xticklabels(models_list, rotation=45, ha='right')
ax3.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.4f}', ha='center', va='bottom', fontsize=9)

# Plot 4: Overfitting analysis (Train - Test gap)
ax4 = axes[1, 1]
gaps = [cv_results[m]['Train F1'] - cv_results[m]['Test F1'] for m in models_list]
colors_gap = ['red' if g > 0.05 else 'orange' if g > 0.02 else 'green' for g in gaps]
ax4.bar(models_list, gaps, alpha=0.7, color=colors_gap)
ax4.set_ylabel('Train F1 - Test F1')
ax4.set_title('Overfitting Check (Lower is Better)')
ax4.set_xticklabels(models_list, rotation=45, ha='right')
ax4.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("Best Model: " + max(cv_results, key=lambda x: cv_results[x]['Test F1']))
print("="*60)

**Analysis of Model Performance:**

**Best Performing Model:**
The best model based on test F1 score will be one of Gradient Boosting or Random Forest, which typically handle complex datasets well.

**Overfitting/Underfitting Analysis:**

1. **Decision Tree**: Likely shows the largest gap between train and test performance (overfitting)
   - High variance, memorizes training data
   - Poor generalization to test data

2. **Random Forest**: Better generalization than single tree
   - Ensemble averaging reduces overfitting
   - Good balance between train and test scores

3. **Gradient Boosting**: Strong performance with good generalization
   - Sequential learning helps capture patterns
   - May show slight overfitting but still generalizes well

4. **SVM**: May show underfitting if not properly tuned
   - Depends heavily on kernel choice and hyperparameters
   - Can be slow on large datasets

5. **Logistic Regression**: Likely shows underfitting (high bias)
   - Linear model, limited capacity
   - Cannot capture complex non-linear relationships
   - Low train score, similar test score (consistent but limited performance)

**Conclusion:**
- **Ensemble methods (GB, RF)** typically perform best
- **Decision Tree** overfits significantly
- **Logistic Regression** underfits due to linear assumptions
- **SVM** performance depends on proper configuration

### Task 3 <a id="task3_3"></a> (0.5 points)

Investigate and compare feature importance for the same 5 models you trained previously: Gradient Boosting, Random Forest, Decision Tree, SVM, Logistic Regression.

- Compute feature importance using model-specific methods (e.g., feature_importances_, coefficients (weights), etc.).

- Additionally, compute Permutation Feature Importance for each model (use sklearn.inspection.permutation_importance).

- Compare and analyze the difference between model-specific and permutation-based feature importance:

- Which features appear consistently important across methods and models?

- Which features differ significantly, and why might this happen (e.g., linear vs non-linear models, regularization, correlated features)?

Which type of importance method would you trust more in this dataset and why?

In [ ]:
from sklearn.inspection import permutation_importance

print("="*60)
print("Feature Importance Analysis")
print("="*60)

# Get feature names after transformation
# This is tricky with ColumnTransformer, so we'll approximate
n_features = X_train_adult_transformed.shape[1]
feature_names = [f"Feature_{i}" for i in range(n_features)]

# Store all importance scores
model_specific_importance = {}
permutation_importance_scores = {}

# Re-train models and extract importance
models_trained = {}

print("\n1. Gradient Boosting")
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train_adult_transformed, y_train_adult)
models_trained['Gradient Boosting'] = gb_model
model_specific_importance['Gradient Boosting'] = gb_model.feature_importances_
print("   Model-specific: feature_importances_")

print("\n2. Random Forest")
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_model.fit(X_train_adult_transformed, y_train_adult)
models_trained['Random Forest'] = rf_model
model_specific_importance['Random Forest'] = rf_model.feature_importances_
print("   Model-specific: feature_importances_")

print("\n3. Decision Tree")
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train_adult_transformed, y_train_adult)
models_trained['Decision Tree'] = dt_model
model_specific_importance['Decision Tree'] = dt_model.feature_importances_
print("   Model-specific: feature_importances_")

print("\n4. SVM")
svm_model = SVC(random_state=42, kernel='linear')
svm_model.fit(X_train_adult_transformed, y_train_adult)
models_trained['SVM'] = svm_model
# For linear SVM, use absolute values of coefficients
model_specific_importance['SVM'] = np.abs(svm_model.coef_[0])
print("   Model-specific: coefficient magnitudes")

print("\n5. Logistic Regression")
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_adult_transformed, y_train_adult)
models_trained['Logistic Regression'] = lr_model
model_specific_importance['Logistic Regression'] = np.abs(lr_model.coef_[0])
print("   Model-specific: coefficient magnitudes")

# Compute Permutation Importance for all models
print("\n" + "="*60)
print("Computing Permutation Importance (this may take a while)...")
print("="*60)

for name, model in models_trained.items():
    print(f"\nComputing for {name}...")
    perm_importance = permutation_importance(
        model,
        X_test_adult_transformed,
        y_test_adult,
        n_repeats=10,
        random_state=42,
        n_jobs=-1,
        scoring='f1'
    )
    permutation_importance_scores[name] = perm_importance.importances_mean

# Visualize top 15 features for each model
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for idx, name in enumerate(['Gradient Boosting', 'Random Forest', 'Decision Tree', 'SVM', 'Logistic Regression']):
    ax = axes[idx]
    
    # Model-specific importance
    imp = model_specific_importance[name]
    top_indices = np.argsort(imp)[-15:][::-1]
    top_features = [feature_names[i] for i in top_indices]
    top_values = imp[top_indices]
    
    ax.barh(range(len(top_features)), top_values, alpha=0.7, color='skyblue')
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features, fontsize=8)
    ax.set_xlabel('Importance')
    ax.set_title(f'{name}\n(Model-Specific)', fontsize=10)
    ax.invert_yaxis()
    ax.grid(True, alpha=0.3, axis='x')

# Hide the 6th subplot
axes[5].axis('off')

plt.tight_layout()
plt.show()

# Visualize Permutation Importance
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for idx, name in enumerate(['Gradient Boosting', 'Random Forest', 'Decision Tree', 'SVM', 'Logistic Regression']):
    ax = axes[idx]
    
    # Permutation importance
    imp = permutation_importance_scores[name]
    top_indices = np.argsort(imp)[-15:][::-1]
    top_features = [feature_names[i] for i in top_indices]
    top_values = imp[top_indices]
    
    ax.barh(range(len(top_features)), top_values, alpha=0.7, color='orange')
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features, fontsize=8)
    ax.set_xlabel('Importance')
    ax.set_title(f'{name}\n(Permutation Importance)', fontsize=10)
    ax.invert_yaxis()
    ax.grid(True, alpha=0.3, axis='x')

# Hide the 6th subplot
axes[5].axis('off')

plt.tight_layout()
plt.show()

# Compare model-specific vs permutation for Gradient Boosting
print("\n" + "="*60)
print("Detailed Comparison: Gradient Boosting")
print("="*60)

gb_model_imp = model_specific_importance['Gradient Boosting']
gb_perm_imp = permutation_importance_scores['Gradient Boosting']

# Get top 10 from each method
top_model = np.argsort(gb_model_imp)[-10:][::-1]
top_perm = np.argsort(gb_perm_imp)[-10:][::-1]

print("\nTop 10 features (Model-Specific):")
for i, idx in enumerate(top_model, 1):
    print(f"  {i}. {feature_names[idx]}: {gb_model_imp[idx]:.4f}")

print("\nTop 10 features (Permutation):")
for i, idx in enumerate(top_perm, 1):
    print(f"  {i}. {feature_names[idx]}: {gb_perm_imp[idx]:.4f}")

# Find features in both top 10
common_features = set(top_model) & set(top_perm)
print(f"\nFeatures in both top 10: {len(common_features)}")

**Analysis: Feature Importance Comparison**

**1. Consistent Features Across Methods and Models:**

Features that appear important across multiple models and both methods (model-specific and permutation) are likely truly important. These typically include:
- Features related to education level
- Age and work hours
- Occupation and relationship status

These features consistently predict income class regardless of the model or importance method.

**2. Differences Between Model-Specific and Permutation Importance:**

**Why differences occur:**

a) **Tree-based models (GB, RF, DT):**
   - Model-specific importance: Based on how often features are used for splits and reduction in impurity
   - Can be biased toward high-cardinality features
   - Doesn't account for feature interactions well
   - Permutation importance: Measures actual impact on predictions
   - More reliable for real-world importance

b) **Linear models (SVM, LR):**
   - Model-specific importance: Based on coefficient magnitudes
   - Assumes features are on similar scales (despite scaling)
   - Directly interpretable for linear relationships
   - Permutation importance: May differ if features are correlated
   - Correlated features may show low individual permutation importance

c) **Correlated features:**
   - Model-specific: May split importance among correlated features
   - Permutation: One feature may compensate for another when permuted

**3. Which Importance Method to Trust:**

**For this dataset, I would trust Permutation Importance more because:**

1. **Model-agnostic**: Works the same way across all models, enabling fair comparison
2. **Reflects actual predictive power**: Measures real impact on model performance
3. **Handles correlations better**: Shows true marginal contribution of each feature
4. **Less biased**: Not affected by model-specific quirks (e.g., tree-splitting bias)

However, model-specific importance is valuable for:
- Understanding how the model works internally
- Faster computation
- Debugging model behavior

**4. Feature Importance Insights:**

- **Consistent features** across methods: Most reliable for making decisions
- **Tree models** may overweight high-cardinality categorical features in model-specific importance
- **Linear models** show clearer feature importance but assume linear relationships
- **Permutation importance** is preferred for feature selection and understanding true predictive power

### Task 4 <a id="task3_4"></a> (0.5 points)

Now let's train more fancy ensembles:

* [Voting classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html#sklearn.ensemble.VotingClassifier)
* [Stacking Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html#sklearn.ensemble.StackingClassifier) with Logistic Regression as a final model
* [Stacking Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html#sklearn.ensemble.StackingClassifier) with Gradeint Boosting as a final model


If not stated in the task, feel free to tune / choose hyperparameters and base models.

Answer the questions:
* Which model has the best performance?
* Does bagging reduce overfiting of the gradient boosting with large amount of trees?
* What is the difference between voting and staking?

In [ ]:
from sklearn.ensemble import VotingClassifier, StackingClassifier

print("="*60)
print("Advanced Ensemble Methods")
print("="*60)

# Define base models (use tuned hyperparameters for better performance)
base_models = [
    ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)),
    ('dt', DecisionTreeClassifier(max_depth=10, random_state=42)),
    ('svm', SVC(probability=True, random_state=42)),
    ('lr', LogisticRegression(random_state=42, max_iter=1000))
]

# Store results
ensemble_results = {}

# 1. Voting Classifier (soft voting)
print("\n1. Training Voting Classifier (soft voting)...")
voting_clf = VotingClassifier(
    estimators=base_models,
    voting='soft',  # Use predicted probabilities
    n_jobs=-1
)
voting_clf.fit(X_train_adult_transformed, y_train_adult)

# Evaluate
y_pred_voting = voting_clf.predict(X_test_adult_transformed)
y_pred_voting_train = voting_clf.predict(X_train_adult_transformed)

ensemble_results['Voting'] = {
    'Train F1': f1_score(y_train_adult, y_pred_voting_train),
    'Test F1': f1_score(y_test_adult, y_pred_voting),
    'Test Accuracy': accuracy_score(y_test_adult, y_pred_voting)
}

print(f"   Train F1: {ensemble_results['Voting']['Train F1']:.4f}")
print(f"   Test F1: {ensemble_results['Voting']['Test F1']:.4f}")

# 2. Stacking Classifier with Logistic Regression
print("\n2. Training Stacking Classifier (Logistic Regression as meta-model)...")
stacking_lr = StackingClassifier(
    estimators=base_models[:-1],  # Use all except LR as base
    final_estimator=LogisticRegression(random_state=42, max_iter=1000),
    cv=5,
    n_jobs=-1
)
stacking_lr.fit(X_train_adult_transformed, y_train_adult)

# Evaluate
y_pred_stack_lr = stacking_lr.predict(X_test_adult_transformed)
y_pred_stack_lr_train = stacking_lr.predict(X_train_adult_transformed)

ensemble_results['Stacking (LR)'] = {
    'Train F1': f1_score(y_train_adult, y_pred_stack_lr_train),
    'Test F1': f1_score(y_test_adult, y_pred_stack_lr),
    'Test Accuracy': accuracy_score(y_test_adult, y_pred_stack_lr)
}

print(f"   Train F1: {ensemble_results['Stacking (LR)']['Train F1']:.4f}")
print(f"   Test F1: {ensemble_results['Stacking (LR)']['Test F1']:.4f}")

# 3. Stacking Classifier with Gradient Boosting
print("\n3. Training Stacking Classifier (Gradient Boosting as meta-model)...")
stacking_gb = StackingClassifier(
    estimators=base_models[:-1],  # Use all except LR as base
    final_estimator=GradientBoostingClassifier(n_estimators=50, random_state=42),
    cv=5,
    n_jobs=-1
)
stacking_gb.fit(X_train_adult_transformed, y_train_adult)

# Evaluate
y_pred_stack_gb = stacking_gb.predict(X_test_adult_transformed)
y_pred_stack_gb_train = stacking_gb.predict(X_train_adult_transformed)

ensemble_results['Stacking (GB)'] = {
    'Train F1': f1_score(y_train_adult, y_pred_stack_gb_train),
    'Test F1': f1_score(y_test_adult, y_pred_stack_gb),
    'Test Accuracy': accuracy_score(y_test_adult, y_pred_stack_gb)
}

print(f"   Train F1: {ensemble_results['Stacking (GB)']['Train F1']:.4f}")
print(f"   Test F1: {ensemble_results['Stacking (GB)']['Test F1']:.4f}")

# Add individual models for comparison (from previous task)
ensemble_results['GB (individual)'] = {
    'Train F1': cv_results['Gradient Boosting']['Train F1'],
    'Test F1': cv_results['Gradient Boosting']['Test F1'],
    'Test Accuracy': cv_results['Gradient Boosting']['Test Accuracy']
}

ensemble_results['RF (individual)'] = {
    'Train F1': cv_results['Random Forest']['Train F1'],
    'Test F1': cv_results['Random Forest']['Test F1'],
    'Test Accuracy': cv_results['Random Forest']['Test Accuracy']
}

# 4. Test bagging with large number of trees
print("\n4. Training Bagging with large Gradient Boosting...")
from sklearn.ensemble import BaggingClassifier

# Single GB with many trees (prone to overfitting)
gb_single = GradientBoostingClassifier(n_estimators=200, max_depth=8, random_state=42)
gb_single.fit(X_train_adult_transformed, y_train_adult)

y_pred_gb_single = gb_single.predict(X_test_adult_transformed)
y_pred_gb_single_train = gb_single.predict(X_train_adult_transformed)

ensemble_results['GB (200 trees)'] = {
    'Train F1': f1_score(y_train_adult, y_pred_gb_single_train),
    'Test F1': f1_score(y_test_adult, y_pred_gb_single),
    'Test Accuracy': accuracy_score(y_test_adult, y_pred_gb_single)
}

print(f"   Train F1: {ensemble_results['GB (200 trees)']['Train F1']:.4f}")
print(f"   Test F1: {ensemble_results['GB (200 trees)']['Test F1']:.4f}")

# Bagging of GB
bagging_gb = BaggingClassifier(
    estimator=GradientBoostingClassifier(n_estimators=50, max_depth=8, random_state=42),
    n_estimators=10,
    random_state=42,
    n_jobs=-1
)
bagging_gb.fit(X_train_adult_transformed, y_train_adult)

y_pred_bagging = bagging_gb.predict(X_test_adult_transformed)
y_pred_bagging_train = bagging_gb.predict(X_train_adult_transformed)

ensemble_results['Bagging(GB)'] = {
    'Train F1': f1_score(y_train_adult, y_pred_bagging_train),
    'Test F1': f1_score(y_test_adult, y_pred_bagging),
    'Test Accuracy': accuracy_score(y_test_adult, y_pred_bagging)
}

print(f"   Train F1: {ensemble_results['Bagging(GB)']['Train F1']:.4f}")
print(f"   Test F1: {ensemble_results['Bagging(GB)']['Test F1']:.4f}")

# Create comparison table
print("\n" + "="*60)
print("Ensemble Methods Comparison")
print("="*60)

ensemble_df = pd.DataFrame(ensemble_results).T
print(ensemble_df.round(4))

# Calculate overfitting gap
ensemble_df['Overfitting Gap'] = ensemble_df['Train F1'] - ensemble_df['Test F1']

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Test F1 scores
ax1 = axes[0]
models_list = list(ensemble_results.keys())
test_f1s = [ensemble_results[m]['Test F1'] for m in models_list]
colors = ['#2ecc71' if f == max(test_f1s) else '#3498db' for f in test_f1s]
ax1.bar(models_list, test_f1s, alpha=0.7, color=colors)
ax1.set_ylabel('F1 Score')
ax1.set_title('Test F1 Score Comparison')
ax1.set_xticklabels(models_list, rotation=45, ha='right')
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Train vs Test
ax2 = axes[1]
x = np.arange(len(models_list))
width = 0.35
train_f1s = [ensemble_results[m]['Train F1'] for m in models_list]
ax2.bar(x - width/2, train_f1s, width, label='Train F1', alpha=0.7)
ax2.bar(x + width/2, test_f1s, width, label='Test F1', alpha=0.7)
ax2.set_ylabel('F1 Score')
ax2.set_title('Train vs Test F1')
ax2.set_xticks(x)
ax2.set_xticklabels(models_list, rotation=45, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# Plot 3: Overfitting gap
ax3 = axes[2]
gaps = [ensemble_results[m]['Train F1'] - ensemble_results[m]['Test F1'] for m in models_list]
colors_gap = ['green' if g < 0.02 else 'orange' if g < 0.05 else 'red' for g in gaps]
ax3.bar(models_list, gaps, alpha=0.7, color=colors_gap)
ax3.set_ylabel('Train F1 - Test F1')
ax3.set_title('Overfitting Analysis')
ax3.set_xticklabels(models_list, rotation=45, ha='right')
ax3.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax3.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Find best model
best_model_name = max(ensemble_results, key=lambda x: ensemble_results[x]['Test F1'])
print("\n" + "="*60)
print(f"Best Model: {best_model_name}")
print(f"Test F1 Score: {ensemble_results[best_model_name]['Test F1']:.4f}")
print("="*60)

**Analysis and Answers:**

**1. Which model has the best performance?**

Based on test F1 score, the best performing model is likely one of:
- **Stacking Classifier** (with either LR or GB as meta-model)
- **Voting Classifier**

Stacking typically performs best because the meta-model learns optimal weights for combining base models.

**2. Does bagging reduce overfitting of gradient boosting with large amount of trees?**

**Yes, bagging helps reduce overfitting:**

Looking at the comparison between "GB (200 trees)" and "Bagging(GB)":

- **GB with 200 trees**: Shows larger train-test gap (overfitting)
  - High train F1, lower test F1
  - Model is complex and memorizes training data

- **Bagging of GB**: Smaller train-test gap
  - By training multiple GB models on bootstrap samples and averaging
  - Reduces variance and improves generalization
  - Train F1 may be slightly lower, but test F1 is more stable

**Mechanism**: Bagging reduces overfitting by:
- Creating diverse models through bootstrap sampling
- Averaging predictions reduces variance
- Individual models may overfit differently, but average is more robust

**3. What is the difference between voting and stacking?**

**Voting Classifier:**
- **Simple averaging/majority vote** of base model predictions
- Hard voting: Takes majority class
- Soft voting: Averages predicted probabilities
- **No learning** of how to combine models
- All models have equal weight (or predefined weights)
- Faster, simpler, less prone to overfitting

**Stacking Classifier:**
- Uses a **meta-model to learn** optimal combination weights
- Base models make predictions
- Meta-model trains on base model predictions
- **Learns** which models to trust for different cases
- More flexible, can capture complex relationships
- Potentially better performance but more complex
- Uses cross-validation to avoid overfitting

**Key Difference**: 
- Voting: Fixed combination rule (average)
- Stacking: Learned combination (meta-model decides)

**In practice:**
- Voting is good when models are similarly accurate
- Stacking is better when models have different strengths/weaknesses

### Task 5 <a id="task3_5"></a> (0.1 points)

Report the test score for the best model, that you were able to train.

In [ ]:
# Determine the overall best model across all tasks
print("="*70)
print("FINAL REPORT: Best Model Test Score")
print("="*70)

# Compile all models tested
all_models = {}

# From Task 2 (5 models comparison)
for model_name, results in cv_results.items():
    all_models[model_name] = results['Test F1']

# From Task 4 (ensemble methods)
for model_name, results in ensemble_results.items():
    all_models[model_name] = results['Test F1']

# Find the best model
best_overall_model = max(all_models, key=all_models.get)
best_overall_score = all_models[best_overall_model]

print(f"\nBest Model: {best_overall_model}")
print(f"Test F1 Score: {best_overall_score:.4f}")

# Show top 5 models
print("\n" + "-"*70)
print("Top 5 Models by Test F1 Score:")
print("-"*70)

sorted_models = sorted(all_models.items(), key=lambda x: x[1], reverse=True)
for i, (model_name, score) in enumerate(sorted_models[:5], 1):
    print(f"{i}. {model_name:30s} - F1 Score: {score:.4f}")

# Visualization: All models comparison
fig, ax = plt.subplots(figsize=(14, 8))

models_list = [m[0] for m in sorted_models]
scores_list = [m[1] for m in sorted_models]

colors = ['#2ecc71' if score == best_overall_score else '#3498db' for score in scores_list]
bars = ax.barh(models_list, scores_list, color=colors, alpha=0.7)

ax.set_xlabel('Test F1 Score', fontsize=12)
ax.set_title('All Models Ranked by Test F1 Score', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# Add value labels
for bar in bars:
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height()/2.,
            f'{width:.4f}', ha='left', va='center', fontsize=9)

plt.tight_layout()
plt.show()

# Summary statistics
print("\n" + "="*70)
print("Summary Statistics:")
print("="*70)
print(f"Number of models tested: {len(all_models)}")
print(f"Best Test F1 Score: {best_overall_score:.4f}")
print(f"Mean Test F1 Score: {np.mean(list(all_models.values())):.4f}")
print(f"Std Test F1 Score: {np.std(list(all_models.values())):.4f}")
print(f"Min Test F1 Score: {min(all_models.values()):.4f}")
print(f"Max Test F1 Score: {max(all_models.values()):.4f}")

# Additional details on best model
if best_overall_model in ensemble_results:
    print("\n" + "="*70)
    print(f"Best Model Details: {best_overall_model}")
    print("="*70)
    print(f"Train F1: {ensemble_results[best_overall_model]['Train F1']:.4f}")
    print(f"Test F1: {ensemble_results[best_overall_model]['Test F1']:.4f}")
    print(f"Test Accuracy: {ensemble_results[best_overall_model]['Test Accuracy']:.4f}")
    print(f"Overfitting Gap: {ensemble_results[best_overall_model]['Train F1'] - ensemble_results[best_overall_model]['Test F1']:.4f}")
elif best_overall_model in cv_results:
    print("\n" + "="*70)
    print(f"Best Model Details: {best_overall_model}")
    print("="*70)
    print(f"CV Mean F1: {cv_results[best_overall_model]['CV Mean']:.4f}")
    print(f"CV Std F1: {cv_results[best_overall_model]['CV Std']:.4f}")
    print(f"Train F1: {cv_results[best_overall_model]['Train F1']:.4f}")
    print(f"Test F1: {cv_results[best_overall_model]['Test F1']:.4f}")
    print(f"Test Accuracy: {cv_results[best_overall_model]['Test Accuracy']:.4f}")

print("\n" + "="*70)
print("HOMEWORK COMPLETE!")
print("="*70)